<a href="https://colab.research.google.com/github/AnnaZhuravleva/compling/blob/master/assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 5

Build CNN model for sentiment analysis (binary classification) of IMDB Reviews (https://www.kaggle.com/utathya/imdb-review-dataset).
You can use data with label="unsup" for pretraining of embeddings. Here you are forbidden to use test dataset for pretraining of embeddings.  
Your quality metric is accuracy score on test dataset. Look at "type" column for  train/test split.  
You can use pretrained embeddings from external sources.  
You have to provide data for trials with different hyperparameter values.  

You have to beat following baselines:  
[3 points] acc = 0.75  
[5 points] acc = 0.8  
[8 points] acc = 0.9  

[2 points] for using unsupervised data  

In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_path = '/content/drive/My Drive/Colab Notebooks/compling'
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
import nltk
import gensim
import spacy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset, Iterator
from keras.utils.np_utils import to_categorical
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn import metrics


SEED = 42
np.random.seed(SEED)

spacy_en = spacy.load('en')

def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text) if tok.text.isalpha()]

Using TensorFlow backend.


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
!head '/content/drive/My Drive/Colab Notebooks/compling/assignment_5/imdb_master.csv'

,type,review,label,file
0,test,"Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.",neg,0_2.txt
1,test,"This is an example of why the majority of acti

In [4]:
import spacy
spacy_en = spacy.load('en')
import nltk
nltk.download('stopwords')

classes = {'neg': 0,
           'pos': 1}
def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text)]

TEXT = Field(include_lengths=True, batch_first=True, 
             tokenize=tokenizer, 
             eos_token='<eos>',
             lower=True,
             stop_words=nltk.corpus.stopwords.words('english'))

LABEL = LabelField(dtype=torch.int64, use_vocab=True, preprocessing=lambda x: classes[x])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
path = '/content/drive/My Drive/Colab Notebooks/compling/assignment_5/imdb_master.csv'
data = pd.read_csv(path, encoding = "ISO-8859-1")
data = data.drop(labels=['Unnamed: 0', 'file'], axis=1)
data.head()

,type,review,label
0,test,Once again Mr. Costner has dragged out a movie...,neg
1,test,This is an example of why the majority of acti...,neg
2,test,"First of all I hate those moronic rappers, who...",neg
3,test,Not even the Beatles could write songs everyon...,neg
4,test,Brass pictures (movies is not a fitting word f...,neg


In [6]:
data.type.unique()

array(['test', 'train'], dtype=object)

In [0]:
train = data[data.type=='train']
train = train[train.label!='unsup'].drop(columns='type')
test = data[data.type=='test'].drop(columns='type')

In [8]:
test.label.unique()

array(['neg', 'pos'], dtype=object)

In [9]:
train.to_csv(path_or_buf='/content/drive/My Drive/Colab Notebooks/compling/assignment_5/train.csv',
            sep='\t',
            index=False)
pd.read_csv('/content/drive/My Drive/Colab Notebooks/compling/assignment_5/train.csv',
            sep='\t').head()

,review,label
0,Story of a man who has unnatural feelings for ...,neg
1,Airport '77 starts as a brand new luxury 747 p...,neg
2,This film lacked something I couldn't put my f...,neg
3,"Sorry everyone,,, I know this is supposed to b...",neg
4,When I was little my parents took me along to ...,neg


In [10]:
test.to_csv(path_or_buf='/content/drive/My Drive/Colab Notebooks/compling/assignment_5/test.csv',
            sep='\t',
            index=False)
pd.read_csv('/content/drive/My Drive/Colab Notebooks/compling/assignment_5/test.csv',
            sep='\t').head()

,review,label
0,Once again Mr. Costner has dragged out a movie...,neg
1,This is an example of why the majority of acti...,neg
2,"First of all I hate those moronic rappers, who...",neg
3,Not even the Beatles could write songs everyon...,neg
4,Brass pictures (movies is not a fitting word f...,neg


In [11]:
test.label.unique()

array(['neg', 'pos'], dtype=object)

In [0]:
dataset = TabularDataset('/content/drive/My Drive/Colab Notebooks/compling/assignment_5/train.csv', format='csv', 
                         fields=[('text', TEXT), ('label', LABEL)], 
                         skip_header=True, csv_reader_params={'delimiter':'\t'})

In [13]:
TEXT.build_vocab(dataset, min_freq=1)
len(TEXT.vocab.itos)

102482

In [0]:
LABEL.build_vocab(dataset)

In [0]:
train, valid = dataset.split(0.7, stratified=True)

In [0]:
testdataset = TabularDataset('/content/drive/My Drive/Colab Notebooks/compling/assignment_5/test.csv', format='csv', 
                         fields=[('text', TEXT), ('label', LABEL)], 
                         skip_header=True, csv_reader_params={'delimiter':'\t'})

In [17]:
np.unique([x.label for x in train.examples], return_counts=True)

(array([0, 1]), array([8750, 8750]))

In [0]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size, kernels):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        self.convs = nn.ModuleList([nn.Conv1d(embed_size, hidden_size, k, padding=5) for k in kernels])
        
        self.fc = nn.Linear(hidden_size * len(kernels), 3)
        
    def forward(self, x):
        
        x = self.embedding(x)
        x = x.transpose(1,2)
        
        concatenated = []
        for conv in self.convs:
            z = conv(x)
            z = F.avg_pool1d(z, kernel_size=z.size(2))
            z = z.squeeze(2)
            concatenated.append(z)
            
        x = torch.cat(concatenated, 1)
        x = self.fc(x)
        return x


In [0]:
# here used materials from assignment 3

def early_stopping(loss, patience=5):
    if len(loss) < patience:
        return False
    best = np.min(np.array(loss))
    return all(map(lambda x: x > best, loss[-patience:]))

def train_batch(model, batch_x, batch_y):
    model.train()
    optimizer.zero_grad()
    pred = model(batch_x)
    loss = criterion(pred, batch_y)
    loss.backward()
    optimizer.step()
    return loss.data.detach().item()

def eval_batch(model, batch_x, batch_y):
    model.eval()
    
    with torch.no_grad():
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
    return loss.data.item()

def train_model(model, train_loader, valid_loader, scheduler, n_epochs, early_stopping_patience=5):
    history = {'train': [], 'valid': []}    
    for epoch in range(n_epochs):
        epoch_train_loss = 0
        for batch in tqdm(train_loader, desc=f'epoch:{epoch}', leave=True):

            loss = train_batch(model, batch.text[0], batch.label)
            epoch_train_loss += loss
            if scheduler:
                scheduler.step(loss)

        # average loss for epoch
        epoch_train_loss /= len(train_loader)
        history['train'].append(epoch_train_loss)
        
        epoch_valid_loss = 0
        for batch in valid_loader:
            loss = eval_batch(model, batch.text[0], batch.label)
            epoch_valid_loss += loss
            
        epoch_valid_loss /= len(valid_loader)
        print(f'train loss: {epoch_train_loss:.3f} valid loss:{epoch_valid_loss:.3f}')
        history['valid'].append(epoch_valid_loss)
        
        if early_stopping(history['valid'], patience=5):
            print('early stopping')
            break

    print('history valid:', history['valid'])

    return history

In [0]:
def test_model(model, test_loader):
    model.eval()
    y_test = []
    y_pred = []
    for batch in tqdm(test_loader, desc='test'):
        with torch.no_grad():
            pred = model(batch.text[0]).cpu()
            y_test.append(batch.label)
            y_pred.append(np.argmax(pred, axis=1))
    y_test = np.concatenate(y_test)
    y_pred = np.concatenate(y_pred)

    return metrics.accuracy_score(y_test, y_pred)

In [21]:
torch.cuda.empty_cache()

batch_size = 32

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
                kernels=[2,3,4,5]
               )

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, testdataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
criterion = nn.CrossEntropyLoss()

history = train_model(model, train_iterator, valid_iterator, scheduler, 3, 2)
test_model(model, test_iterator)

Epoch    13: reducing learning rate of group 0 to 1.0000e-04.
Epoch    24: reducing learning rate of group 0 to 1.0000e-05.
Epoch    35: reducing learning rate of group 0 to 1.0000e-06.
Epoch    46: reducing learning rate of group 0 to 1.0000e-07.
Epoch    57: reducing learning rate of group 0 to 1.0000e-08.

train loss: 0.709 valid loss:0.947



train loss: 0.705 valid loss:0.947



train loss: 0.704 valid loss:0.947
history valid: [0.947454129127746, 0.9473906273537493, 0.9473189412279331]


0.49856

In [24]:
torch.cuda.empty_cache()

batch_size = 32

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
                kernels=[2,3,4,5]
               )

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, testdataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
criterion = nn.CrossEntropyLoss()

history = train_model(model, train_iterator, valid_iterator, None, 3, 2)
test_model(model, test_iterator)


train loss: 0.676 valid loss:0.577



train loss: 0.444 valid loss:0.560



train loss: 0.303 valid loss:0.662
history valid: [0.5772334365451589, 0.5604494630101513, 0.6619749065488577]


0.864

In [26]:
torch.cuda.empty_cache()

batch_size = 32

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
                kernels=[2,3,4,5]
               )

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, testdataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
criterion = nn.CrossEntropyLoss()

history = train_model(model, train_iterator, valid_iterator, None, 1, 2)
test_model(model, test_iterator)


train loss: 0.693 valid loss:0.486
history valid: [0.4855578999332291]


0.86468

In [28]:
torch.cuda.empty_cache()

batch_size = 32

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
                kernels=[2,3,4,5]
               )

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, testdataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
criterion = nn.CrossEntropyLoss()

history = train_model(model, train_iterator, valid_iterator, None, 2, 2)
test_model(model, test_iterator)

train loss: 0.678 valid loss:0.519


train loss: 0.214 valid loss:0.808
history valid: [0.5191468948954122, 0.8077740054379435]


0.87576

In [0]:
torch.cuda.empty_cache()

batch_size = 32

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
                kernels=[2,3,4,5]
               )

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, testdataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True, cooldown=5)
criterion = nn.CrossEntropyLoss()

history = train_model(model, train_iterator, valid_iterator, None, 10, 5)
test_model(model, test_iterator)